- UDAF 는 스칼라와 자바로만 사용할 수 있음
~~~
inputSchema : UDAF 입력 파라미터의 스키마를 StructType으로 정의
bufferSchema : UDAF 중간 결과를 스키마를 StuctType으로 정의
dataType : 반환될 값의 DataType을 정의
deterministic : UDAF 가 동일한 입력값에 대해 항상 동일한 결과를 반환하는지 불리언값으로 정의
initialize : 집계용 버퍼의 값을 초기화하는 로직을 정의
update : 입력받은 로우를 기반으로 내부 버퍼를 업데이트하는 로직을 정의
merge : 두 개의 집계용 버퍼를 병합하는 로직을 정의
evaluate : 집계의 최동 결과를 생성하는 로직을 정의
~~~

In [2]:
%scala

import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql._
import org.apache.spark.sql.types._

class BoolAnd extends UserDefinedAggregateFunction{
  def inputSchema: StructType = 
  StructType(StructField("value", BooleanType) :: Nil) // Nill 빈 리스트
  def bufferSchema: StructType = 
  StructType(StructField("result", BooleanType) :: Nil)
  def dataType: DataType = BooleanType
  def deterministic: Boolean = true
  def initialize(buffer: MutableAggregationBuffer): Unit = { // Unit은 void개념
    buffer(0) = true
  }
  def update(buffer: MutableAggregationBuffer, input: Row):Unit = {
    buffer(0) = buffer.getAs[Boolean](0) && input.getAs[Boolean](0)
  }
  def merge(buffer1: MutableAggregationBuffer, buffer2 : Row) : Unit = {
    buffer1(0) = buffer1.getAs[Boolean](0) && buffer2.getAs[Boolean](0)
  }
  def evaluate(buffer: Row): Any = {
    buffer(0)
  }
}

import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql._
import org.apache.spark.sql.types._
defined class BoolAnd

In [3]:
%scala

val ba = new BoolAnd

spark.udf.register("booland", ba)
import org.apache.spark.sql.functions._

spark.range(1)
  .selectExpr("explode(array(TRUE, TRUE, TRUE)) as t") // 모든 컬럼이 True이면 true
  .selectExpr("explode(array(TRUE, FALSE, TRUE)) as f", "t") // 모든 컬럼이 true이면 false
  .select(ba(col("t")), expr("booland(f)")).show()

+----------+----------+
booland(t)|booland(f)|
+----------+----------+
 true| false|
+----------+----------+

ba: BoolAnd = BoolAnd@42641a84
import org.apache.spark.sql.functions._